# Tranformer model with PyTorch

This notebook creates and trains a tranformer with encoder-decoder architecture using PyTorch

In [1]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
import math
import json
from keras.preprocessing.sequence import pad_sequences

from spotify import SpotifyClient

__Creating the model__

Creating the layers for model. The multi-head attention layer

In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output
    
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))
    
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

__Encoder Layer__

In [5]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

__Decoder Layer__

In [6]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [7]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

__Loading in the data from the challenge set__

In [8]:
# define a batch size for our experiments
BATCH_SIZE = 35
# define a percentage of the data to use for training
SPLIT_PC = .90

# open the file and convert it to json
f = open('spotify_million_playlist_dataset_challenge/challenge_set.json')
js = json.load(f)
playlists = js['playlists']

titles = []
tracks = []

# process and add the playlist names and tracks to lists
for playlist in playlists:
    if not playlist['tracks'] or 'name' not in playlist:
        continue
    titles.append(playlist['name'].lower()) 
    tracks.append(' '.join(track['track_uri'] for track in playlist['tracks']))

END = int(len(titles)*SPLIT_PC)

In [9]:
# custom tokenizer for separating tracks (just separates by spaces)
# we tried using tokenizers from libraries, but they split up the track URIs, which we didn't want
class Tokenizer:
    def __init__(self):
        self.dictionary = {}
        self.reverse_dictionary = {}

        # Add the padding token
        self.dictionary['<UNK>'] = 0
        self.reverse_dictionary[0] = '<UNK>'
        self.cur_token = 1

    def fit_on_texts(self, sentences):
        for sentence in sentences:
            self.tokenize(sentence)

    def tokenize(self, text):
        for token in text.split():
            if token not in self.dictionary:
                self.dictionary[token] = self.cur_token
                self.reverse_dictionary[self.cur_token] = token
                self.cur_token += 1

    def character_to_token(self, character):
        return self.dictionary[character]

    def token_to_character(self, token):
        return self.reverse_dictionary[token]
    
    def texts_to_sequences(self, sentences):
        sequences = []
        for sentence in sentences:
            cur_sentence = []
            for token in sentence.split():
                if token not in self.dictionary:
                    cur_sentence.append(self.dictionary['<UNK>'])
                else:
                    cur_sentence.append(self.dictionary[token])
            sequences.append(cur_sentence)
        return sequences

    def size(self):
        return len(self.dictionary)

__Tokenize and pad titles and tracks__

In [10]:
# tokenizes the given sentences and returns a list of lists of the tokens
def tokenize(sentences):
    text_tokenizer = Tokenizer()
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [11]:
# tokenize titles and tracks, and also get the tokenizers
titles_tokens, title_tokenizer = tokenize(titles)
tracks_tokens, track_tokenizer = tokenize(tracks)
print(titles_tokens[:10])

title_vocab = title_tokenizer.size() + 1
track_vocab = track_tokenizer.size() + 1
print(f'Vocabularies: {title_vocab}, {track_vocab}')

max_title_length = int(len(max(titles_tokens, key=len)))
max_track_length = int(len(max(tracks_tokens, key=len)))
print(f'Max lengths: {max_title_length}, {max_track_length}')

pad_titles = pad_sequences(titles_tokens, max_title_length, padding = "post")
pad_tracks = pad_sequences(tracks_tokens, max_track_length, padding = "post")
print(pad_titles[:10])

title_tensor = torch.LongTensor(pad_titles)
tracks_tensor = torch.LongTensor(pad_tracks)
print(f'Tensor shapes: {title_tensor.shape}, {tracks_tensor.shape}')


[[1], [2], [3, 4], [5, 6], [7], [8, 9], [10], [11], [12, 13], [14]]
Vocabularies: 2487, 63998
Max lengths: 9, 100
[[ 1  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  0  0]
 [ 3  4  0  0  0  0  0  0  0]
 [ 5  6  0  0  0  0  0  0  0]
 [ 7  0  0  0  0  0  0  0  0]
 [ 8  9  0  0  0  0  0  0  0]
 [10  0  0  0  0  0  0  0  0]
 [11  0  0  0  0  0  0  0  0]
 [12 13  0  0  0  0  0  0  0]
 [14  0  0  0  0  0  0  0  0]]
Tensor shapes: torch.Size([7000, 9]), torch.Size([7000, 100])


__Create a data generator for training__

In [12]:
# data generator for title and track tensors
def data_generator(titles: torch.LongTensor, tracks: torch.LongTensor, batch_size: int = BATCH_SIZE):
    i = 0
    while True:
        # return the current batch
        yield titles[i: i + batch_size], tracks[i: i + batch_size]
        
        i += batch_size
        # if we reach the end of the tensor, start from the beginning again
        if i >= len(titles):
            i = 0

In [13]:
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

# initialize a Transformer model (from above) using the declared params
transformer = Transformer(title_vocab, track_vocab, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)


__Training__

This took about 30 sec for 10 epochs with batch size of 35 on a M2 Macbook Air

In [14]:
epochs = 10

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()
generator = data_generator(title_tensor, tracks_tensor)

for epoch in range(epochs):
    optimizer.zero_grad()
    titles, tracks = next(generator)
    output = transformer(titles, tracks)
    loss = criterion(output.contiguous().view(-1, track_vocab), tracks.contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 11.17990493774414
Epoch: 2, Loss: 11.23021125793457
Epoch: 3, Loss: 11.131558418273926
Epoch: 4, Loss: 11.186010360717773
Epoch: 5, Loss: 11.110666275024414
Epoch: 6, Loss: 11.137246131896973
Epoch: 7, Loss: 11.171707153320312
Epoch: 8, Loss: 10.95988655090332
Epoch: 9, Loss: 11.113736152648926
Epoch: 10, Loss: 10.92345905303955


__Evaluating and generating predictions__

In [28]:
# Change the model to eval mode
transformer.eval()
output = transformer(title_tensor[1:2], tracks_tensor[1:2])

# Get the track with the highest probability, then convert the tokens back to URIs
output = output.view(-1, track_vocab).argmax(1)
output = [track_tokenizer.token_to_character(i) for i in output.numpy()]
print(output)

output = list(set(output))

# use Spotify API to get the song names
client = SpotifyClient()
client.get_song_titles(output)

{'tracks': [{'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0Y5tJX1MQlPlqiwlOH1tJY'},
      'href': 'https://api.spotify.com/v1/artists/0Y5tJX1MQlPlqiwlOH1tJY',
      'id': '0Y5tJX1MQlPlqiwlOH1tJY',
      'name': 'Travis Scott',
      'type': 'artist',
      'uri': 'spotify:artist:0Y5tJX1MQlPlqiwlOH1tJY'}],
    'available_markets': ['AR',
     'AU',
     'AT',
     'BE',
     'BO',
     'BR',
     'BG',
     'CA',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DK',
     'DO',
     'DE',
     'EC',
     'EE',
     'SV',
     'FI',
     'FR',
     'GR',
     'GT',
     'HN',
     'HK',
     'HU',
     'IS',
     'IE',
     'IT',
     'LV',
     'LT',
     'LU',
     'MY',
     'MT',
     'MX',
     'NL',
     'NZ',
     'NI',
     'NO',
     'PA',
     'PY',
     'PE',
     'PH',
     'PL',
     'PT',
     'SG',
     'SK',
     'ES',
     'SE',
     'CH',
     'TW',
     'TR',
     'UY',
     'US',
     'GB',
